In [33]:
#basic packages 
import pandas as pd
import numpy as np
import random
import math
from scipy import stats
import pickle 
from itertools import combinations
from collections import Counter
import sqlite3
#plot
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
import plotly.express as px
import plotly.graph_objects as pg
from plotly import tools
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook_connected"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
user_df = pd.read_pickle('Cleaned Data/User.pkl')
transaction_df = pd.read_pickle('Cleaned Data/Transaction.pkl')
product_df = pd.read_pickle('Cleaned Data/Product.pkl')

In [35]:
file_paths = {
    "Product": "Cleaned Data/Product.pkl",
    "User": "Cleaned Data/User.pkl",
    "Transaction": "Cleaned Data/Transaction.pkl"
}
fetch = sqlite3.connect("Cleaned_Data.db")
for table_name, file_path in file_paths.items():
    df = pd.read_pickle(file_path)
    df.to_sql(table_name, fetch, if_exists="replace", index=False)
    print(f"Table '{table_name}' successfully written to the database.")

Table 'Product' successfully written to the database.
Table 'User' successfully written to the database.
Table 'Transaction' successfully written to the database.


##### <span style='color:Blue'> **II. OA Questions** </span>

> <span style='color:Blue'> **II.1. Closed-ended Questions.**</span>

In [36]:
# Check schema for the 'Product' table
cursor = fetch.cursor()
cursor.execute("PRAGMA table_info(Product);")
product_schema = cursor.fetchall()

print("Product Table Schema:")
for column in product_schema:
    print(f"Column Name: {column[1]}, Data Type: {column[2]}")

Product Table Schema:
Column Name: CATEGORY_1, Data Type: TEXT
Column Name: CATEGORY_2, Data Type: TEXT
Column Name: CATEGORY_3, Data Type: TEXT
Column Name: CATEGORY_4, Data Type: TEXT
Column Name: MANUFACTURER, Data Type: TEXT
Column Name: BRAND, Data Type: TEXT
Column Name: BARCODE, Data Type: REAL


In [37]:
cursor.execute("PRAGMA table_info(User);")
user_schema = cursor.fetchall()

print("\nUser Table Schema:")
for column in user_schema:
    print(f"Column Name: {column[1]}, Data Type: {column[2]}")


User Table Schema:
Column Name: ID, Data Type: TEXT
Column Name: CREATED_DATE, Data Type: TIMESTAMP
Column Name: BIRTH_DATE, Data Type: TIMESTAMP
Column Name: STATE, Data Type: TEXT
Column Name: LANGUAGE, Data Type: TEXT
Column Name: GENDER, Data Type: TEXT


In [38]:
cursor.execute("PRAGMA table_info('Transaction');")
transaction_schema = cursor.fetchall()

print("\nTransaction Table Schema:")
for column in transaction_schema:
    print(f"Column Name: {column[1]}, Data Type: {column[2]}")


Transaction Table Schema:
Column Name: RECEIPT_ID, Data Type: TEXT
Column Name: PURCHASE_DATE, Data Type: TIMESTAMP
Column Name: SCAN_DATE, Data Type: TIMESTAMP
Column Name: STORE_NAME, Data Type: TEXT
Column Name: USER_ID, Data Type: TEXT
Column Name: BARCODE, Data Type: REAL
Column Name: FINAL_QUANTITY, Data Type: REAL
Column Name: FINAL_SALE, Data Type: REAL


> What are the top 5 brands by receipts scanned among users 21 and over?

In [39]:
query = """
WITH AdultUsers AS (
    SELECT ID
    FROM User
    WHERE julianday('now') - julianday(BIRTH_DATE) >= 21 * 365 
),
BrandReceipts AS (
    SELECT p.BRAND, COUNT(t.RECEIPT_ID) AS receipt_count
    FROM 'Transaction' t
    JOIN Product p ON t.BARCODE = p.BARCODE
    JOIN AdultUsers u ON t.USER_ID = u.ID
    WHERE p.BRAND IS NOT NULL 
    GROUP BY p.BRAND
)
SELECT BRAND, receipt_count
FROM BrandReceipts
ORDER BY receipt_count DESC
LIMIT 5;
"""
cursor.execute(query)
top_5_brands = cursor.fetchall()
print("Top 5 Brands by Receipts Scanned among Users 21 and Over:")
for brand, receipt_count in top_5_brands:
    print(f"Brand: {brand}, Receipts Scanned: {receipt_count}")


Top 5 Brands by Receipts Scanned among Users 21 and Over:
Brand: DOVE, Receipts Scanned: 3
Brand: NERDS CANDY, Receipts Scanned: 3
Brand: COCA-COLA, Receipts Scanned: 2
Brand: GREAT VALUE, Receipts Scanned: 2
Brand: HERSHEY'S, Receipts Scanned: 2


    >Python Approach

In [40]:
# Filter users who are 21 and over
user_df['AGE'] = (pd.to_datetime('today').tz_localize('UTC') - user_df['BIRTH_DATE']).dt.days // 365
adult_users = user_df[user_df['AGE'] >= 21]

# Merge tbles
merged_transaction = pd.merge(transaction_df, adult_users[['ID']], how='inner', left_on='USER_ID', right_on='ID')
merged_data = pd.merge(merged_transaction, product_df[['BARCODE', 'BRAND']], how='inner', on='BARCODE')

# Filter out rows where BRAND is None or NaN
merged_data = merged_data[merged_data['BRAND'].notna()]

# Group by BRAND and count the number of RECEIPT_ID (i.e., the number of transactions per brand)
brand_receipt_count = merged_data.groupby('BRAND')['RECEIPT_ID'].count().reset_index()
top_5_brands = brand_receipt_count.sort_values(by='RECEIPT_ID', ascending=False).head(5)

print("Top 5 Brands by Receipts Scanned among Users 21 and Over (Excluding None Values):")
print(top_5_brands)

Top 5 Brands by Receipts Scanned among Users 21 and Over (Excluding None Values):
              BRAND  RECEIPT_ID
35      NERDS CANDY           3
13             DOVE           3
51  SOUR PATCH KIDS           2
56          TRIDENT           2
32           MEIJER           2


> What are the top 5 brands by sales among users that have had their account for at least six months?

In [41]:
query = """
WITH LongTermUsers AS (
    SELECT ID
    FROM User
    WHERE julianday('now') - julianday(CREATED_DATE) >= 180
),
BrandSales AS (
    SELECT p.BRAND, SUM(t.FINAL_SALE) AS total_sales
    FROM 'Transaction' t
    JOIN Product p ON t.BARCODE = p.BARCODE
    JOIN LongTermUsers u ON t.USER_ID = u.ID
    WHERE p.BRAND IS NOT NULL 
    GROUP BY p.BRAND
)
SELECT BRAND, total_sales
FROM BrandSales
ORDER BY total_sales DESC
LIMIT 5;
"""

# Execute the query
cursor.execute(query)

# Fetch the results
top_5_brands_by_sales = cursor.fetchall()

# Print the results
print("Top 5 Brands by Sales among Users with Accounts for At Least Six Months (Excluding None Values):")
for brand, total_sales in top_5_brands_by_sales:
    print(f"Brand: {brand}, Total Sales: {total_sales}")


Top 5 Brands by Sales among Users with Accounts for At Least Six Months (Excluding None Values):
Brand: CVS, Total Sales: 72.0
Brand: DOVE, Total Sales: 30.91
Brand: TRIDENT, Total Sales: 23.36
Brand: COORS LIGHT, Total Sales: 17.48
Brand: TRESEMMÉ, Total Sales: 14.58


    > Python Approach

In [42]:
current_date = pd.to_datetime('today').tz_localize('UTC')
user_df['account_age_months'] = (current_date - user_df['CREATED_DATE']).dt.days / 30

# Filter users who have had an account for at least 6 months
long_term_users_df = user_df[user_df['account_age_months'] >= 6]
print(long_term_users_df[['ID', 'account_age_months']].head())

                         ID  account_age_months
0  5ef3b4f17053ab141787697d           55.733333
1  5ff220d383fcfc12622b96bc           49.300000
2  6477950aa55bb77a0e27ee10           20.033333
3  658a306e99b40f103b63ccf8           13.066667
4  653cf5d6a225ea102b7ecdc2           15.033333


In [43]:
transaction_filtered = transaction_df[transaction_df['USER_ID'].isin(long_term_users_df['ID'])]
merged_df = pd.merge(transaction_filtered, product_df[['BARCODE', 'BRAND']], on='BARCODE', how='inner')

# Group by BRAND and calculate total sales
brand_sales = merged_df.groupby('BRAND')['FINAL_SALE'].sum().reset_index()
top_5_brands_by_sales = brand_sales.sort_values(by='FINAL_SALE', ascending=False).head(5)
print(top_5_brands_by_sales)

          BRAND  FINAL_SALE
9           CVS       72.00
13         DOVE       30.91
56      TRIDENT       23.36
7   COORS LIGHT       17.48
55     TRESEMMÉ       14.58


> <span style='color:Blue'> **II.2. Open-ended Questions.**</span>

> **Who are Fetch’s power users?**

- Power User Definition:

    - Frequent Transactions: A user is considered a power user if their transaction count is greater than the average number of transactions across all users.

    - High Spending: A user is considered a power user if their total spending is greater than the average total spending across all users.
    
    - Diverse Product Range: A user is considered a power user if they have bought from more product categories than the average number of categories purchased across all users.

In [44]:
cursor.execute("""
    WITH user_sales AS (
        SELECT t.USER_ID,
            COUNT(t.RECEIPT_ID) AS total_transactions,
            SUM(t.FINAL_SALE) AS total_spending,
            COUNT(DISTINCT p.CATEGORY_1) AS categories_bought
        FROM 'Transaction' t
        JOIN Product p ON t.BARCODE = p.BARCODE
        GROUP BY t.USER_ID
    ),
    averages AS (
        SELECT 
            AVG(total_transactions) AS avg_transactions,
            AVG(total_spending) AS avg_spending,
            AVG(categories_bought) AS avg_categories
        FROM user_sales
    )
    SELECT 
        us.USER_ID,
        us.total_transactions,
        us.total_spending,
        us.categories_bought
    FROM user_sales us, averages a
    WHERE 
        us.total_transactions > a.avg_transactions
        AND us.total_spending > a.avg_spending
        AND us.categories_bought > a.avg_categories
    ORDER BY us.total_transactions DESC, us.total_spending DESC, us.categories_bought DESC
    LIMIT 10
""")

filtered_users = cursor.fetchall()
print("Power Users (Above Average in Transactions, Spending, and Categories):")
for user in filtered_users:
    user_id, total_transactions, total_spending, categories_bought = user
    print(f"User ID: {user_id}")
    print(f"Total Transactions: {total_transactions}")
    print(f"Total Spending: ${total_spending}")
    print(f"Categories Purchased From: {categories_bought}\n")


Power Users (Above Average in Transactions, Spending, and Categories):
User ID: 64063c8880552327897186a5
Total Transactions: 7
Total Spending: $28.669999999999998
Categories Purchased From: 3

User ID: 5e89fe8918bf1a13ef5d874c
Total Transactions: 5
Total Spending: $28.830000000000002
Categories Purchased From: 2

User ID: 62b6189d37e6e08b0774ce73
Total Transactions: 5
Total Spending: $20.97
Categories Purchased From: 3

User ID: 62e6f1ce48cc274645652f44
Total Transactions: 5
Total Spending: $20.11
Categories Purchased From: 3

User ID: 60a42b33f29c34057f5e46a9
Total Transactions: 5
Total Spending: $18.77
Categories Purchased From: 2

User ID: 62925c1be942f00613f7365e
Total Transactions: 5
Total Spending: $15.64
Categories Purchased From: 2

User ID: 5f906764542e501679d188f4
Total Transactions: 4
Total Spending: $52.93
Categories Purchased From: 2

User ID: 60316ede521dd912dc83e2d6
Total Transactions: 4
Total Spending: $38.26
Categories Purchased From: 2

User ID: 64432ac1ee6ac839f8c934

> Which is the leading brand in the Dips & Salsa category?

- Leading Brand Definition:

The leading brand in the Dips & Salsa category is defined as the brand that has generated the highest total sales (sum of FINAL_SALE) within the category. This metric reflects the brand's sales performance and customer preference in this product segment.

In the case of a tie in sales, we could extend this definition to consider other metrics such as number of unique users who purchased from the brand, but given there were not many common user ID between Transaction and User table, we are focusing purely on total sales.

In [45]:
for col in product_df.select_dtypes(include=['object']).columns:
    if product_df[col].str.contains("Dips & Salsa", na=False).any():
        print(f"'Dips & Salsa' found in column: {col}")

'Dips & Salsa' found in column: CATEGORY_2


In [46]:
leading_brand_query = """
    SELECT 
        p.BRAND, 
        SUM(t.FINAL_SALE) AS total_sales
    FROM 'Transaction' t
    JOIN Product p ON t.BARCODE = p.BARCODE
    WHERE p.CATEGORY_2 = 'Dips & Salsa'  
    GROUP BY p.BRAND
    ORDER BY total_sales DESC
    LIMIT 5
"""

cursor.execute(leading_brand_query)
leading_brands = cursor.fetchall()  # Fetch all rows
if leading_brands:
    print("Top 5 leading brands in the Dips & Salsa category:")
    for brand, total_sales in leading_brands:
        print(f"{brand}: ${total_sales:.2f}")
else:
    print("No brands found in the Dips & Salsa category.")


Top 5 leading brands in the Dips & Salsa category:
TOSTITOS: $181.30
None: $100.97
GOOD FOODS: $94.91
PACE: $85.75
FRITOS: $67.16


In [47]:
fetch.close()